# SPANISH NEWS CLASSIFIER
## WIP NEED LARGER SAMPLE

In [ ]:
!pip install pattern

     |████████████████████████████████| 22.2 MB 77.7 MB/s 
     |████████████████████████████████| 87 kB 6.4 MB/s 
     |████████████████████████████████| 81 kB 8.2 MB/s 
     |████████████████████████████████| 5.6 MB 22.9 MB/s 
     |████████████████████████████████| 5.6 MB 36.2 MB/s 
     |████████████████████████████████| 419 kB 59.5 MB/s 
     |████████████████████████████████| 104 kB 52.1 MB/s 
     |████████████████████████████████| 3.6 MB 48.9 MB/s 
  Created wheel for pattern: filename=Pattern-3.6-py3-none-any.whl size=22332721 sha256=9fe0c1f9a288ee0d77cdac6a5c67403a5bb9b1659e37fc279b1da4932912930e
  Stored in directory: /root/.cache/pip/wheels/8d/1f/4e/9b67afd2430d55dee90bd57618dd7d899f1323e5852c465682
  Created wheel for mysqlclient: filename=mysqlclient-2.1.0-cp37-cp37m-linux_x86_64.whl size=99971 sha256=fded56e3ad9d2a2a99cec3fdb28001584bcc122cdab4c959593a5289488e534c
  Stored in directory: /root/.cache/pip/wheels/97/d4/df/08cd6e1fa4a8691b268ab254bd0fa589827ab5b65638c010b4
 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# Libraries for text preprocessing
import re
import nltk

#nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer#nltk.download('wordnet') 
from nltk.stem.wordnet import WordNetLemmatizer
import os

### Libraries
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import string
from nltk.stem.snowball import SnowballStemmer
englishStemm=SnowballStemmer("english").stem
spanishStemm=SnowballStemmer("spanish").stem
import pattern
from pattern.es import lemma as spanishlemma
from pattern.en import lemma as englishlemma

nltk.download('stopwords')
from nltk.corpus import stopwords
stpwrd = stopwords.words('spanish')
stpwrd.extend("toggle") # THE TRAINING DATA IS LITTERED WITH THIS, SO I USE THIS STOPWORD TO REMOVE IT
stopEnglish=set(stopwords.words('english'))
stopSpanish=set(stpwrd)

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV,train_test_split
import time
import joblib

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/Portfolios/data_larazon_publico_v2.csv', delimiter=',', header=0)
df = df.fillna(0)
df = df.drop(columns=['Unnamed: 0', 'indi'])

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
class preprocesaTexto():
    def __init__(self, idioma,_tokeniza=False,_aMinusculas=True,
                 _aMayusculas=False,_removerPuntuacion=True,_removerCaracteresRaros=True,
                 _quitarNumeros=True,_lematiza=False,_stemming=False,_quitarAcentos=False,
                 _remueveStop=False,_muestraCambios=False):
      self.idioma=idioma
      self._tokeniza=_tokeniza
      self._aMinusculas=_aMinusculas
      self._aMayusculas=_aMayusculas
      self._removerPuntuacion=_removerPuntuacion
      self._removerCaracteresRaros=_removerCaracteresRaros
      self._quitarNumeros=_quitarNumeros
      self._lematiza=_lematiza
      self._stemming=_stemming
      self._quitarAcentos =_quitarAcentos
      self._remueveStop =_remueveStop
      self._muestraCambios=_muestraCambios
        
    def tokeniza(self,texto):
      return word_tokenize(texto)

    def aMinusculas(self,texto):
      if self.idioma== 'es':
        ans=""
        for c in texto:
          if c=='Á' : c='á'
          if c=='É' : c='é'
          if c=='Í' : c='í'
          if c=='Ó' : c='ó'
          if c=='Ú' : c='ú'
          if c=='Ñ' : c='ñ'
          if c=='Ü' : c='ü'
          ans+=c
        return ans.lower()
      else :
        return texto.lower()

    def aMayusculas(self,texto):
      if self.idioma == 'es':
        ans=""
        for c in texto:
          if c=='á' : c='Á'
          if c=='é' : c='É'
          if c=='í' : c='Í'
          if c=='ó' : c='Ó'
          if c=='ú' : c='Ú'
          if c=='ñ' : c='Ñ'
          if c=='ü' : c='Ü'
          ans+=c
        return ans.upper()
      else :
        return texto.upper()

    def quitarAcentos(self,texto):
      if self.idioma == 'es':
        ans=""
        for c in texto:
          if c=='á' : c='a'
          if c=='é' : c='e'
          if c=='í' : c='i'
          if c=='ó' : c='o'
          if c=='ú' : c='u'
          if c=='Á' : c='A'
          if c=='É' : c='E'
          if c=='Í' : c='I'
          if c=='Ó' : c='O'
          if c=='Ú' : c='U'
          ans+=c
        return ans
      else :
        return texto

    def removerPuntuacion(self,texto):
      puntuacion=string.punctuation
      if self.idioma=='es': 
        puntuacion+="¡¿"
      temp=""
      for c in texto:
        if not c in puntuacion:
          temp+=c
      return temp

    def removerCaracteresRaros(self,texto):
      buenos=string.printable
      if self.idioma=='es': 
        buenos+="¡¿áéíóúÁÉÍÓÚÑñÜü"
      temp=""
      for c in texto:
        if c in buenos:
          temp+=c
      return temp

    def quitarNumeros(self,texto):
      temp=""
      for c in texto:
        if not c.isdigit():
          temp+=c
      return temp

    def lematiza(self,texto):
      palabras=word_tokenize(texto)
      temp=[]
      if self.idioma== 'es':
        for pal in palabras:
          temp.append(spanishlemma(pal))
          temp.append(" ")
      else :
        for pal in palabras:
          temp.append(englishlemma(pal))
          temp.append(" ")
      return "".join(temp)

    def stemming(self,texto):
      palabras=word_tokenize(texto)
      temp=[]
      if self.idioma== 'es':
        for pal in palabras:
          temp.append(spanishStemm(pal))
          temp.append(" ")
      else :
        for pal in palabras:
          temp.append(englishStemm(pal))
          temp.append(" ")
      return "".join(temp)

    def remueveStop(self,texto):
      palabras=word_tokenize(texto)
      temp=[]
      if self.idioma== 'es':
        for pal in palabras:
          if pal not in stopSpanish:
            temp.append(pal)
            temp.append(" ")
      else :
        for pal in palabras:
          if pal not in stopEnglish:
            temp.append(pal)
            temp.append(" ")
      return "".join(temp)

    def preprocesa(self,texto):
      ans=texto
      if self._muestraCambios : print("Inicial:\n", ans)
      if self._removerCaracteresRaros : 
        ans=self.removerCaracteresRaros(ans)
        if self._muestraCambios : print("Quitando caracteres raros:\n", ans)
      if self._removerPuntuacion : 
        ans=self.removerPuntuacion(ans)
        if self._muestraCambios :print("Quitando signos de puntuacion:\n", ans)
      if self._quitarNumeros : 
        ans=self.quitarNumeros(ans)
        if self._muestraCambios : print("Quitando numeros:\n", ans)
      if self._aMinusculas : 
        ans=self.aMinusculas(ans)
        if self._muestraCambios : print("Convirtiendo a minusculas:\n", ans)  
      if self._aMayusculas : 
        ans=self.aMayusculas(ans)
        if self._muestraCambios : print("Convirtiendo a mayusculas:\n", ans)
      if self._remueveStop : 
        ans=self.remueveStop(ans)
        if self._muestraCambios : print("Quitando stop words:\n", ans)
      if self._quitarAcentos : 
        ans=self.quitarAcentos(ans)
        if self._muestraCambios : print("Quitando acentos:\n", ans)
      if self._lematiza : 
        ans=self.lematiza(ans)
        if self._muestraCambios : print("Lematizando:\n", ans)
      if self._stemming : 
        ans=self.stemming(ans)
        if self._muestraCambios : print("Stemming:\n", ans)
      if self._tokeniza : 
        ans=self.tokeniza(ans)
        if self._muestraCambios : print("Tokenizando:\n", ans)
      if self._muestraCambios : print("Final:\n", ans)
      return ans 

preprocesador = preprocesaTexto(idioma='es',_tokeniza=False,
                                _muestraCambios=False,_quitarAcentos=True,
                                _remueveStop=True,_stemming=False)

# NEWS CLASSIFICATION SUPERVISED

In [ ]:
X_train = df.iloc[:,1]
y_train = df.iloc[:,0]

In [16]:
# Read training data
from google.colab import drive
drive.mount('/content/drive')
TrainingData = pd.read_csv('/content/drive/MyDrive/Portfolios/Training_data.csv')
X_train = TrainingData.iloc[:,1]
y_train = TrainingData.iloc[:,0]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# #############################################################################
# Define a pipeline combining a text feature extractor with a simple
# classifier
pipeline = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=39, max_iter=5, tol=None)), ])


# find the best parameters for both the feature extraction and the
# classifier
# uncommenting more parameters will give better exploring power but will
# increase processing time in a combinatorial way
parameters = {
    "vect__max_df": (0.5, 0.75, 1.0),
    # 'vect__max_features': (None, 5000, 10000, 50000),
    "vect__ngram_range": ((1, 1), (1, 2)),  # unigrams or bigrams
    # 'tfidf__use_idf': (True, False),
    # 'tfidf__norm': ('l1', 'l2'),
    "clf__max_iter": (10,),
    "clf__alpha": (0.00001, 0.000001),
    "clf__penalty": ("l2", "elasticnet"),
    # 'clf__max_iter': (10, 50, 80),
}

# find the best parameters for both the feature extraction and the
# classifier
grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1)

print("Performing grid search...")
print("parameters:")
print(parameters)
start_time = time.time()
grid_search.fit(X_train, y_train)

print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))
    print((time.time() - start_time))

Performing grid search...
parameters:
{'vect__max_df': (0.5, 0.75, 1.0), 'vect__ngram_range': ((1, 1), (1, 2)), 'clf__max_iter': (10,), 'clf__alpha': (1e-05, 1e-06), 'clf__penalty': ('l2', 'elasticnet')}
Fitting 5 folds for each of 24 candidates, totalling 120 fits


In [ ]:
# Best parameter estimator
pipeline2 = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), 
                      ('clf', SGDClassifier(loss='hinge', penalty='elasticnet',alpha=1e-5, random_state=39, max_iter=10, tol=None)), ])

# Fit the best SGDClassifier to train data
pipeline2.fit(X_train, y_train)

import joblib
# save the model to disk
filename = 'finalized_model.sav'
joblib.dump(pipeline2, filename)

# load the model from disk
loaded_model = joblib.load(filename)

In [ ]:
# Predict topics SGDClassifier
data= pd.read_csv("data/Ejercicio_notas.csv")
corpus2 = []
for i in range(0, len(data['texto'])):
    corpus2.append(preprocesador.preprocesa(data['texto'][i]))
# Fit the best SGDClassifier to train data
loaded_model.predict(corpus2)